# loss

> An implementation of the loss function for training the [YOLOX](https://arxiv.org/abs/2107.08430) object detection model based on [OpenMMLab](https://github.com/open-mmlab)’s implementation in the [mmdetection](https://github.com/open-mmlab/mmdetection) library.

In [ ]:
#| default_exp loss

In [ ]:
#| hide
from nbdev.showdoc import *

In [ ]:
#| export
from typing import Any, Type, List, Optional, Callable, Tuple, Union
from functools import partial

In [ ]:
#| export
import numpy as np

import torch
import torch.nn.functional as F
import torchvision

In [ ]:
#| export
from cjm_yolox_pytorch.utils import multi_apply, generate_grid_priors
from cjm_yolox_pytorch.simota import SimOTAAssigner

In [ ]:
#| export
class SamplingResult:
    """
    Bounding box sampling result.
    
    Based on OpenMMLab's implementation in the mmdetection library:
    
    - [OpenMMLab's Implementation](https://github.com/open-mmlab/mmdetection/blob/d64e719172335fa3d7a757a2a3636bd19e9efb62/mmdet/core/bbox/samplers/sampling_result.py#L7)

    """
    def __init__(self, 
                 positive_indices:np.array, # Indices of the positive samples.
                 negative_indices:np.array, # Indices of the negative samples.
                 bboxes:np.array, # Array containing all bounding boxes.
                 ground_truth_bboxes:torch.Tensor, # Tensor containing all ground truth bounding boxes.
                 assignment_result, # Object that contains the ground truth indices and labels corresponding to each sample.
                 ground_truth_flags:np.array # Array indicating which samples are ground truth.
                ):
        # Indices of positive and negative samples
        self.positive_indices = positive_indices
        self.negative_indices = negative_indices

        # Bounding boxes for positive and negative samples
        self.positive_bboxes = bboxes[positive_indices]
        self.negative_bboxes = bboxes[negative_indices]
        
        # Flag indicating if the positive samples are ground truth
        self.is_positive_ground_truth = ground_truth_flags[positive_indices]

        # Number of ground truths
        self.number_of_ground_truths = ground_truth_bboxes.shape[0]
        
        # Indices of the assigned ground truths for positive samples
        self.positive_assigned_ground_truth_indices = assignment_result.ground_truth_box_indices[positive_indices] - 1

        # Check the consistency of ground truth bounding boxes and assigned indices
        if ground_truth_bboxes.numel() == 0:
            if self.positive_assigned_ground_truth_indices.numel() != 0:
                raise ValueError('Mismatch between ground truth bounding boxes and positive assigned ground truth indices.')
            self.positive_ground_truth_bboxes = torch.empty_like(ground_truth_bboxes).view(-1, 4)
        else:
            if len(ground_truth_bboxes.shape) < 2:
                ground_truth_bboxes = ground_truth_bboxes.view(-1, 4)
            self.positive_ground_truth_bboxes = ground_truth_bboxes[self.positive_assigned_ground_truth_indices, :]

        # If labels are assigned, assign labels for positive samples. Otherwise, set it as None
        if assignment_result.category_labels is not None:
            self.positive_ground_truth_labels = assignment_result.category_labels[positive_indices]
        else:
            self.positive_ground_truth_labels = None

In [ ]:
#| export
class YOLOXLoss:
    """
    YOLOXLoss class implements the loss function used in the YOLOX model. 
    
    Based on OpenMMLab's implementation in the mmdetection library:
    
    - [OpenMMLab's Implementation](https://github.com/open-mmlab/mmdetection/blob/d64e719172335fa3d7a757a2a3636bd19e9efb62/mmdet/models/dense_heads/yolox_head.py#L321)

    #### Pseudocode
    1. Generate the grid of prior boxes by calling the `generate_grid_priors` function with arguments based on the shape of class scores and strides.
    2. Update the centroids of the prior boxes based on their widths and heights.
    3. Flatten the predicted class scores, bounding box predictions, and objectness scores across all scales and concatenate them. This is done by calling the `flatten_and_concat` method for each of these predictions.
    4. Decode the predicted bounding boxes by calling the `bbox_decode` method, which calculates the actual coordinates of the predicted boxes based on the prior boxes and the predicted bounding box transformations.
    5. For each image in the batch, compute the targets for classification, bounding box regression, objectness, and optionally, L1 regression. This is done by calling the `get_target_single` method, which matches prior boxes to ground truth boxes, samples positive and negative boxes, and then computes the targets. 
    6. Calculate the total number of positive samples across all images in the batch.
    7. Concatenate all the computed targets across all images.
    8. If the `use_l1` flag is set to `True`, concatenate the computed L1 targets across all images.
    9. Compute the bounding box, objectness, and class losses by comparing the predictions with the targets. 
    10. Multiply the computed losses by their respective weights.
    11. Create a dictionary to store the computed losses.
        
    """
    def __init__(self, 
                 num_classes:int, # The number of target classes.
                 bbox_loss_weight:float=5.0, # The weight for the loss function to calculate the bounding box regression loss.
                 class_loss_weight:float=1.0, # The weight for the loss function to calculate the classification loss.
                 objectness_loss_weight:float=1.0, # The weight for the loss function to calculate the objectness loss.
                 l1_loss_weight:float=1.0, # The weight for the loss function to calculate the L1 loss.
                 use_l1:bool=False, # Whether to use L1 loss in the calculation.
                 strides:List[int]=[8,16,32] # The list of strides.
                ):
        
        self.num_classes = num_classes
        
        giou_loss_partial = partial(torchvision.ops.generalized_box_iou_loss, reduction='none', eps=1e-16)
        self.bbox_loss_func = lambda bx1, bx2 : (1-(1-giou_loss_partial(boxes1=bx1, boxes2=bx2))**2).sum()
        self.class_loss_func = partial(F.binary_cross_entropy_with_logits, reduction='sum')
        self.objectness_loss_func = partial(F.binary_cross_entropy_with_logits, reduction='sum')
        self.l1_loss_func = partial(F.l1_loss, reduction='sum')
        
        self.bbox_loss_weight = bbox_loss_weight
        self.class_loss_weight = class_loss_weight
        self.objectness_loss_weight = objectness_loss_weight
        self.l1_loss_weight = l1_loss_weight
        
        self.use_l1 = use_l1
        
        # Initialize the assigner
        self.assigner = SimOTAAssigner(center_radius=2.5)
        
        self.strides = strides
        
        
    def bbox_decode(self, prior_boxes, predicted_boxes):
        """
        Decodes the predicted bounding boxes based on the prior boxes.

        Args:
            prior_boxes (torch.Tensor): The prior bounding boxes.
            predicted_boxes (torch.Tensor): The predicted bounding boxes.

        Returns:
            torch.Tensor: The decoded bounding boxes.
        """
        # Calculate box centroids (geometric centers) and sizes
        box_centroids = (predicted_boxes[..., :2] * prior_boxes[:, 2:]) + prior_boxes[:, :2]
        box_sizes = torch.exp(predicted_boxes[..., 2:]) * prior_boxes[:, 2:]

        # Calculate corners of bounding boxes
        top_left = box_centroids - box_sizes / 2
        bottom_right = box_centroids + box_sizes / 2

        # Stack coordinates to create decoded bounding boxes
        decoded_boxes = torch.cat((top_left, bottom_right), -1)

        return decoded_boxes


    def sample(self, assignment_result, bboxes, ground_truth_boxes, **kwargs):
        """
        Samples positive and negative indices based on the assignment result.
        
        Args:
            assignment_result (Object): The assignment result obtained from assigner.
            bboxes (torch.Tensor): The predicted bounding boxes.
            ground_truth_boxes (torch.Tensor): The ground truth boxes.

        Returns:
            SamplingResult: The sampling result containing positive and negative indices.
        """
        positive_indices = torch.nonzero(
            assignment_result.ground_truth_box_indices > 0, as_tuple=False).squeeze(-1).unique()
        negative_indices = torch.nonzero(
            assignment_result.ground_truth_box_indices == 0, as_tuple=False).squeeze(-1).unique()
        ground_truth_flags = bboxes.new_zeros(bboxes.shape[0], dtype=torch.uint8)
        sampling_result = SamplingResult(positive_indices, negative_indices, bboxes, ground_truth_boxes,
                                         assignment_result, ground_truth_flags)
        return sampling_result


    def get_l1_target(self, l1_target, ground_truth_boxes, prior_boxes, epsilon=1e-8):
        """
        Calculates the L1 target.

        Args:
            l1_target (torch.Tensor): The L1 target tensor.
            ground_truth_boxes (torch.Tensor): The ground truth boxes.
            prior_boxes (torch.Tensor): The prior bounding boxes.
            epsilon (float, optional): A small value to prevent division by zero. Defaults to 1e-8.

        Returns:
            torch.Tensor: The updated L1 target.
        """
        ground_truth_centroid_and_wh = torchvision.ops.box_convert(ground_truth_boxes, 'xyxy', 'cxcywh')
        l1_target[:, :2] = (ground_truth_centroid_and_wh[:, :2] - prior_boxes[:, :2]) / prior_boxes[:, 2:]
        l1_target[:, 2:] = torch.log(ground_truth_centroid_and_wh[:, 2:] / prior_boxes[:, 2:] + epsilon)
        return l1_target


    def get_target_single(self, class_preds, objectness_score, prior_boxes, decoded_bboxes,
                           ground_truth_bboxes, ground_truth_labels):
        """
        Calculates the targets for a single image.

        Args:
            class_preds (torch.Tensor): The predicted class probabilities.
            objectness_score (torch.Tensor): The predicted objectness scores.
            prior_boxes (torch.Tensor): The prior bounding boxes.
            decoded_bboxes (torch.Tensor): The decoded bounding boxes.
            ground_truth_bboxes (torch.Tensor): The ground truth boxes.
            ground_truth_labels (torch.Tensor): The ground truth labels.

        Returns:
            Tuple: The targets for classification, objectness, bounding boxes, and L1 (if applicable), along with
            the foreground mask and the number of positive samples.
        """
        # Get the number of prior boxes and ground truth labels
        num_prior_boxes = prior_boxes.size(0)
        num_ground_truths = ground_truth_labels.size(0)

        # Match dtype of ground truth bounding boxes to the dtype of decoded bounding boxes
        ground_truth_bboxes = ground_truth_bboxes.to(decoded_bboxes.dtype)

        # Check if there are no ground truth labels (objects) in the image
        if num_ground_truths == 0:
            # Initialize targets as zero tensors, and foreground_mask as a boolean tensor with False values
            class_targets = class_preds.new_zeros((0, self.num_classes))
            bbox_targets = class_preds.new_zeros((0, 4))
            l1_targets = class_preds.new_zeros((0, 4))
            objectness_targets = class_preds.new_zeros((num_prior_boxes, 1))
            foreground_mask = class_preds.new_zeros(num_prior_boxes).bool()
            return (foreground_mask, class_targets, objectness_targets, bbox_targets,
                    l1_targets, 0)  # Return zero for num_positive_per_image

        # Calculate the offset for the prior boxes
        offset_prior_boxes = torch.cat([prior_boxes[:, :2] + prior_boxes[:, 2:] * 0.5, prior_boxes[:, 2:]], dim=-1)

        # Assign ground truth objects to prior boxes and get assignment results
        assignment_result = self.assigner.assign(
            class_preds.sigmoid() * objectness_score.unsqueeze(1).sigmoid(),
            offset_prior_boxes, decoded_bboxes, ground_truth_bboxes, ground_truth_labels)

        # Use assignment results to sample prior boxes
        sampling_result = self.sample(assignment_result, prior_boxes, ground_truth_bboxes)
        
        # Get the indices of positive (object-containing) samples
        positive_indices = sampling_result.positive_indices
        num_positive_per_image = positive_indices.size(0)

        # Get the maximum IoU values for the positive samples
        positive_ious = assignment_result.max_iou_values[positive_indices]

        # Generate class targets
        class_targets = F.one_hot(sampling_result.positive_ground_truth_labels, self.num_classes) * positive_ious.unsqueeze(-1)

        # Initialize objectness targets as zeros and set the values at positive_indices to 1
        objectness_targets = torch.zeros_like(objectness_score).unsqueeze(-1)
        objectness_targets[positive_indices] = 1

        # Generate bounding box targets
        bbox_targets = sampling_result.positive_ground_truth_bboxes

        # Initialize L1 targets as zeros
        l1_targets = class_preds.new_zeros((num_positive_per_image, 4))

        # If use_l1 is True, calculate L1 targets
        if self.use_l1:
            l1_targets = self.get_l1_target(l1_targets, bbox_targets, prior_boxes[positive_indices])

        # Initialize foreground_mask as zeros and set the values at positive_indices to True
        foreground_mask = torch.zeros_like(objectness_score).to(torch.bool)
        foreground_mask[positive_indices] = 1

        # Return the computed targets, the foreground mask, and the number of positive samples per image
        return (foreground_mask, class_targets, objectness_targets, bbox_targets, l1_targets, num_positive_per_image)
    
    
    def flatten_and_concat(self, tensors, num_images, reshape_dims=None):
        new_shape = (num_images, -1, reshape_dims) if reshape_dims else (num_images, -1)
        return torch.cat([t.permute(0, 2, 3, 1).reshape(*new_shape) for t in tensors], dim=1)

    
    def __call__(self, num_images, class_scores, predicted_bboxes, objectness_scores, ground_truth_bboxes, ground_truth_labels):
        """
        Main method to compute the YOLOX loss.

        Args:
            num_images (int): The number of images in a batch.
            class_scores (List[torch.Tensor]): A list of class scores for each scale.
            predicted_bboxes (List[torch.Tensor]): A list of predicted bounding boxes for each scale.
            objectness_scores (List[torch.Tensor]): A list of objectness scores for each scale.
            ground_truth_bboxes (List[torch.Tensor]): A list of ground truth bounding boxes for each image.
            ground_truth_labels (List[torch.Tensor]): A list of ground truth labels for each image.

        Returns:
            Dict: A dictionary with the classification, bounding box, objectness, and optionally, L1 loss.
        """
        
        # Generate prior box coordinates for all anchors.
        grid_priors = generate_grid_priors(*[s*self.strides[0] for s in class_scores[0].shape[-2:]], self.strides)
        grid_priors[:, :2] *= grid_priors[:, 2].unsqueeze(1)
        flatten_prior_boxes = torch.cat([grid_priors, grid_priors[:, 2:].clone()], dim=1)
        
        # Flatten and concatenate class predictions, bounding box predictions, and objectness scores
        flatten_class_preds = self.flatten_and_concat(class_scores, num_images, self.num_classes)
        flatten_bbox_preds = self.flatten_and_concat(predicted_bboxes, num_images, 4)
        flatten_objectness_scores = self.flatten_and_concat(objectness_scores, num_images)
                    
        # Concatenate and decode box predictions
        flatten_prior_boxes = flatten_prior_boxes.to(flatten_bbox_preds.device)
        flatten_decoded_bboxes = self.bbox_decode(flatten_prior_boxes, flatten_bbox_preds)

        # Compute targets
        (positive_masks, class_targets, objectness_targets, bbox_targets, l1_targets,
         num_positive_images) = multi_apply(
             self.get_target_single, flatten_class_preds.detach(),
             flatten_objectness_scores.detach(),
             flatten_prior_boxes.unsqueeze(0).repeat(num_images, 1, 1),
             flatten_decoded_bboxes.detach(), ground_truth_bboxes, ground_truth_labels)

        # Calculate total number of samples
        num_total_samples = max(sum(num_positive_images), 1)

        # Concatenate all positive masks, class targets, objectness targets, and bounding box targets
        positive_masks = torch.cat(positive_masks, 0)
        class_targets = torch.cat(class_targets, 0)
        objectness_targets = torch.cat(objectness_targets, 0)
        bbox_targets = torch.cat(bbox_targets, 0)

        # Compute bounding box loss
        loss_bbox = self.bbox_loss_func(flatten_decoded_bboxes.view(-1, 4)[positive_masks], bbox_targets)

        # Compute objectness loss
        loss_obj = self.objectness_loss_func(flatten_objectness_scores.view(-1, 1), objectness_targets)

        # Compute class loss
        loss_cls = self.class_loss_func(flatten_class_preds.view(-1, self.num_classes)[positive_masks],class_targets)
                
        # Scale losses
        loss_bbox = (loss_bbox * self.bbox_loss_weight) / num_total_samples
        loss_obj = (loss_obj * self.objectness_loss_weight) / num_total_samples
        loss_cls = (loss_cls * self.class_loss_weight) / num_total_samples
        
        # Initialize loss dictionary
        loss_dict = dict(loss_cls=loss_cls, loss_bbox=loss_bbox, loss_obj=loss_obj)

        # If use_l1 is True, concatenate l1 targets, compute L1 loss and add it to the loss dictionary
        if self.use_l1:
            l1_targets = torch.cat(l1_targets, 0)
            loss_l1 = self.l1_loss_func(
                flatten_bbox_preds.view(-1, 4)[positive_masks],
                l1_targets) / num_total_samples
            loss_l1 *= self.l1_loss_weight
            loss_dict.update(loss_l1=loss_l1)

        # Return loss dictionary
        return loss_dict

In [ ]:
#| hide
import nbdev; nbdev.nbdev_export()